# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K
from keras.regularizers import l2


import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random
from keras.layers import GaussianNoise

# Loading data

In [ ]:
tr_pairs = np.load('features_pca_train_pairs_normalized.npy')
train_y = np.load('features_pca_train_y_norm.npy')
train_y=train_y.astype('float32') 

ts_pairs = np.load('features_pca_test_pairs_normalized.npy')
test_y = np.load('features_pca_test_y_norm.npy') 
test_y=test_y.astype('float32')

# Base Network

In [ ]:
with tf.device('/CPU:0'):
        
    def initialize_base_network():
        
        input = Input(shape=(4000), name="base_input")
        
        x = Dense(512, activation='relu', name="first_base_dense")(input)
        x = Dropout(0.3, name="first_dropout")(x)
        x=GaussianNoise(0.3)(x)
        x = Dense(512, activation='relu', name="second_base_dense")(x)
        x = Dropout(0.3, name="second_dropout")(x)
        x = Dense(512, activation='relu', name="third_base_dense")(x)
       
    
        return Model(inputs=input, outputs=x)
    
    
    def euclidean_distance(vects):
        x, y = vects
        sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_square, K.epsilon()))
    
    
    def eucl_dist_output_shape(shapes):
        shape1, shape2 = shapes
        return (shape1[0], 1)
    
    
    base_network = initialize_base_network()
    
    # create the left input and point to the base network
    input_a = Input(shape=(4000), name="left_input")
    vect_output_a = base_network(input_a)
    
    # create the right input and point to the base network
    input_b = Input(shape=(4000), name="right_input")
    vect_output_b = base_network(input_b)
    
    # measure the similarity of the two vector outputs
    output = Lambda(euclidean_distance, name="output_layer", output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])
    
    # specify the inputs and output of the model
    model = Model([input_a, input_b], output)

# Contrastive loss

In [ ]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss

# Custom metric function

In [ ]:
def custom_acc(y_true, y_pred):
    pred = y_pred < 0.5
    x=tf.cast(y_true,tf.bool)  
    return K.mean(pred == x)

class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('custom_acc') > 0.96):
        
          self.model.stop_training = True

# Training

In [ ]:
with tf.device('/CPU:0'):
    
    rms = RMSprop()
    opt=Adam()
    model.compile(loss = contrastive_loss_with_margin(margin =1), optimizer = opt, metrics = [custom_acc])
    callbackss = myCallback()
    history = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], train_y, epochs = 200, batch_size = 128, validation_data=([ts_pairs[:, 0], ts_pairs[:, 1]], test_y),callbacks=[callbackss])   